# Generating C code for the right-hand sides of Maxwell's equations in Cartesian Coordinates

This tutorial will draw on previous work done by Ian Ruchlin on [Maxwell's equations in Cartesian Coordinates](Tutorial-MaxwellCurvilinear.ipynb), which itself drew on the two formulations described in [Illustrating Stability Properties of Numerical Relativity in Electrodynamics](https://arxiv.org/abs/gr-qc/0201051).This will be done to aid construction of an Einstein Toolkit thorn, which will itself be built in [the next tutorial](Tutorial-ETK_thorn-Maxwell.ipynb).

The construction of our equations here will be nearly identical; however, by assuming Cartesian coordinates, we are able to make several simplifications and eliminate the need for [reference_metric.py](../edit/reference_metric.py) as a dependency.

We will begin with their System I. While the Curvilinear version of this code assumed flat spacetime. 

One particular difficulty we will encounter here is taking the covariant Laplacian of a vector. This will here take the form of $D_j D^j A_i$. We will start with the outer derivative after lowering the index on the second operator. So, we see that 
\begin{align}
D_j D^j A_i &= \gamma^{jk} D_j D_k A_i \\
& = \gamma^{jk} [\partial_j (D_k A_i) - \Gamma^l_{ij} D_k A_l - \Gamma^l_{jk} D_l A_i].
\end{align}
Next, we will again apply the covariant derivative to $A_i$. First, however, we should consider that 
\begin{align}
D_k A_i &= \partial_k A_i - \Gamma^l_{ik} A_l \\
& = \partial_k A_i - \gamma^{lm} \Gamma_{mjk} A_l \\
& = \partial_k A_i - \Gamma_{mjk} A^m, \\
\end{align}
where $\Gamma_{ljk} = \frac{1}{2} (\partial_k \gamma_{lj} + \partial_j \gamma_{kl} - \partial_l \gamma_{jk})$ is the Christoffel symbol of the first kind. Note how we were able to use the raising operator to switch the height of two indices; we will use this in upcoming steps. Thus, our expression becomes
\begin{align}
D_j D^j A_i &= \gamma^{jk} [\partial_j \partial_k A_i - \partial_j (\Gamma_{lik} A^l) - \Gamma^l_{ij} \partial_k A_l + \Gamma^m_{ij} \Gamma_{lmk} A^l - \Gamma^l_{jk} \partial_l A_i + \Gamma^m_{jk} \Gamma_{lim} A^l] \\
&= \gamma^{jk} [\partial_j \partial_k A_i - \underbrace{\partial_j (\Gamma_{lik} A^l)}_{\text{sub-term}} - \Gamma^l_{ij} \partial_k A_l + \Gamma^m_{ij} \Gamma^l_{mk} A_l - \Gamma^l_{jk} \partial_l A_i + \Gamma^m_{jk} \Gamma^l_{im} A_l].
\end{align}
Let's focus on the underbraced sub-term for a moment. Expanding this using the product rule and the definition of the Christoffel symbol, 
\begin{align}
\partial_j (\Gamma_{lik} A^l) &= A^l \partial_j \Gamma_{lik} + \Gamma_{lik} \partial_j A^l \\
&= A^l \partial_j (\partial_k \gamma_{li} + \partial_i \gamma_{kl} - \partial_l \gamma_{ik}) + \Gamma_{lik} \partial_j (\gamma^{lm} A_m) \\
&= A^l (\gamma_{li,kj} + \gamma_{kl,ij} - \gamma_{ik,lj}) + \Gamma_{lik} (\gamma^{lm} A_{m,j} + A_m \gamma^{lm}{}_{,j}), \\
\end{align}
where commas in subscripts denote partial derivatives.

So, the Divergence becomes 
\begin{align}
D_j D^j A_i &= \gamma^{jk} [A_{i,jk} - 
\underbrace{A^l (\gamma_{li,kj} + \gamma_{kl,ij} - \gamma_{ik,lj})}_{\text{Term 1}} + 
\underbrace{\Gamma_{lik} (\gamma^{lm} A_{m,j} + A_m \gamma^{lm}{}_{,j})}_{\text{Term 2}} - 
\underbrace{(\Gamma^l_{ij} A_{l,k} + \Gamma^l_{jk} A_{i,l})}_{\text{Term 3}} + 
\underbrace{(\Gamma^m_{ij} \Gamma_{lmk} A^l + \Gamma ^m_{jk} \Gamma^l_{im} A_l)}_{\text{Term 4}}]; \\
\end{align}
we will now begin to contruct these terms individually.

In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

# Step 2: Register gridfunctions that are needed as input.
psi = gri.register_gridfunctions("EVOL", ["psi"])

# Step 3a: Declare the rank-1 indexed expressions E_{i}, A_{i},
#          and \partial_{i} \psi. Derivative variables like these
#          must have an underscore in them, so the finite
#          difference module can parse the variable name properly.
ED = ixp.register_gridfunctions_for_single_rank1("EVOL", "ED")
AD = ixp.register_gridfunctions_for_single_rank1("EVOL", "AD")
psi_dD = ixp.declarerank1("psi_dD")
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])

## Step 3b: Declare the conformal metric tensor and its first 
#           derivative. These are needed to find the Christoffel
#           symbols, which we need for covariant derivatives.
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym12") # The AUX or EVOL designation is *not*
                                                                                # used in diagnostic modules.
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym12")

gammaUU = ixp.declarerank3("gammaUU","sym12")
detgamma = gri.register_gridfunctions("AUX",["detgamma"])
gammaUU, detgamma = ixp.symm_matrix_inverter3x3(gammaDD)
gammaUU_dD = ixp.declarerank3("gammaDD_dD","sym12")

# Define the Christoffel symbols
GammaUDD = ixp.zerorank3(DIM)
for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                GammaUDD[i][k][l] += (sp.Rational(1,2))*gammaUU[i][m]*\
                                     (gammaDD_dD[m][k][l] + gammaDD_dD[m][l][k] - gammaDD_dD[k][l][m])

GammaUDD_dD = ixp.declarerank4("GammaUDD_dD","none")

# Step 3b: Declare the rank-2 indexed expression \partial_{j} A_{i},
#          which is not symmetric in its indices.
#          Derivative variables like these must have an underscore
#          in them, so the finite difference module can parse the
#          variable name properly.
AD_dD = ixp.declarerank2("AD_dD", "nosym")

# Step 3c: Declare the rank-3 indexed expression \partial_{jk} A_{i},
#          which is symmetric in the two {jk} indices.
AD_dDD = ixp.declarerank3("AD_dDD", "sym23")

# Step 4: Calculate first and second covariant derivatives, and the
#         necessary contractions.
# First covariant derivative
# D_{j} A_{i} = A_{i,j} - \Gamma^{k}_{ij} A_{k}
AD_dHatD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        AD_dHatD[i][j] = AD_dD[i][j]
        for k in range(DIM):
            AD_dHatD[i][j] -= GammaUDD[k][i][j] * AD[k]

# Second covariant derivative
# D_{k} D_{j} A_{i} = \partial_{k} D_{j} A_{i} - \Gamma^{l}_{jk} D_{l} A_{i}
#                    - \Gamma^{l}_{ik} D_{j} A_{l}
#                   =  A_{i,jk}
#                    - \Gamma^{l}_{ij,k} A_{l} 
#                    - \Gamma^{l}_{ij}   A_{l,k} 
#                    - \Gamma^{l}_{jk}   A_{i;\hat{l}}
#                    - \Gamma^{l}_{ik}   A_{l;\hat{j}}
AD_dHatDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            AD_dHatDD[i][j][k] = AD_dDD[i][j][k]
            for l in range(DIM):
                AD_dHatDD[i][j][k] += - GammaUDD_dD[l][i][j][k]  * AD[l]  \
                                      - GammaUDD[l][i][j] *    AD_dD[l][k] \
                                      - GammaUDD[l][j][k] * AD_dHatD[i][l] \
                                      - GammaUDD[l][i][k] * AD_dHatD[l][j]

# Covariant divergence
# D_{i} A^{i} = \gamma^{ij} D_{j} A_{i}
DivA = 0
# Gradient of covariant divergence
# DivA_dD_{i} = \gamma^{jk} A_{k;\hat{j}\hat{i}}
DivA_dD = ixp.zerorank1()
# Covariant Laplacian
# LapAD_{i} = \gamma^{jk} A_{i;\hat{j}\hat{k}}
LapAD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        DivA += gammaUU[i][j] * AD_dHatD[i][j]
        for k in range(DIM):
            DivA_dD[i] += gammaUU[j][k] * AD_dHatDD[k][j][i]
            LapAD[i]   += gammaUU[j][k] * AD_dHatDD[i][j][k]

# Step 5: Define right-hand sides for the evolution.
AD_rhs = ixp.zerorank1()
ED_rhs = ixp.zerorank1()
for i in range(DIM):
    AD_rhs[i] = -ED[i] - psi_dD[i]
    ED_rhs[i] = -LapAD[i] + DivA_dD[i]
psi_rhs = -DivA
    
# Step 6: Generate C code for System I Maxwell's evolution equations,
#         print output to the screen (standard out, or stdout).
lhrh_list = []
for i in range(DIM):
    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "AD" + str(i)), rhs=AD_rhs[i]))
    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "ED" + str(i)), rhs=ED_rhs[i]))
lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "psi"), rhs=psi_rhs))
    
fin.FD_outputC("stdout", lhrh_list)

Error: Attempting to take the derivative of GammaUDD221, which is not a registered gridfunction.


NameError: name 'exit' is not defined

We will now build the equations for System II.

In [ ]:
# We inherit here all of the definitions from System I, above

# Step 7a: Register the scalar auxiliary variable \Gamma
Gamma = gri.register_gridfunctions("EVOL", ["Gamma"])

# Step 7b: Declare the ordinary gradient \partial_{i} \Gamma
Gamma_dD = ixp.declarerank1("Gamma_dD")

# Step 8a: Construct the second covariant derivative of the scalar \psi
# \psi_{;\hat{i}\hat{j}} = \psi_{,i;\hat{j}}
#                        = \psi_{,ij} - \Gamma^{k}_{ij} \psi_{,k}
psi_dDD = ixp.declarerank2("psi_dDD", "sym12")
psi_dHatDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        psi_dHatDD[i][j] = psi_dDD[i][j]
        for k in range(DIM):
            psi_dHatDD[i][j] += - rfm.GammahatUDD[k][i][j] * psi_dD[k]

# Step 8b: Construct the covariant Laplacian of \psi
# Lappsi = ghat^{ij} D_{j} D_{i} \psi
Lappsi = 0
for i in range(DIM):
    for j in range(DIM):
        Lappsi += rfm.ghatUU[i][j] * psi_dHatDD[i][j]

# Step 9: Define right-hand sides for the evolution.
AD_rhs = ixp.zerorank1()
ED_rhs = ixp.zerorank1()
for i in range(DIM):
    AD_rhs[i] = -ED[i] - psi_dD[i]
    ED_rhs[i] = -LapAD[i] + Gamma_dD[i]
psi_rhs = -Gamma
Gamma_rhs = -Lappsi
    
# Step 10: Generate C code for System II Maxwell's evolution equations,
#          print output to the screen (standard out, or stdout).
lhrh_list = []
for i in range(DIM):
    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "AD" + str(i)), rhs=AD_rhs[i]))
    lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "ED" + str(i)), rhs=ED_rhs[i]))
lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "psi"), rhs=psi_rhs))
lhrh_list.append(lhrh(lhs=gri.gfaccess("rhs_gfs", "Gamma"), rhs=Gamma_rhs))
    
fin.FD_outputC("stdout", lhrh_list)